In [5]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from network import CharacterClassifier
from tqdm import tqdm
from training_data import HASY
from PIL import Image
from matplotlib import pyplot as plt

hasy_train = HASY('HASY')
hasy_test = HASY('HASY', train=False)

print("Train data length: {0}".format(len(hasy_train.data)))
print("Test data length: {0}".format(len(hasy_test.data)))
print("Img Shape: {0}".format(hasy_train.data[0].shape))
print("Number of Labels: {0}".format(hasy_train.no_labels))

100%|██████████| 16992/16992 [00:00<00:00, 219509.95it/s]

Train data length: 1958
Test data length: 224
Img Shape: torch.Size([1, 28, 28])
Number of Labels: 15


In [6]:
import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from network import CharacterClassifier
from tqdm import tqdm
from training_data import HASY
from PIL import Image
from matplotlib import pyplot as plt
from training_data import OwnImgs
from torchvision import models
from torch.nn import Conv2d

# Training Iteration with our images
model = models.densenet201(num_classes=15)
model.features.conv0 = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
own_imgs = OwnImgs()

print("Train data length: {0}".format(len(own_imgs.data)))
print("Img Shape: {0}".format(own_imgs.data[0].shape))
print("Number of Labels: {0}".format(own_imgs.no_labels))

img_loader = DataLoader(own_imgs, batch_size=16, shuffle=True)
model.load_state_dict(torch.load('combined-model.ckpt'))

epochs = 15
optimizer = torch.optim.Adam(hasy_model.parameters(), lr=0.001, weight_decay=0.001)
criterion = nn.CrossEntropyLoss()
for epoch in range(epochs):
    print("Epoch {0}".format(epoch))
    for step, [x_train, y_train] in enumerate(tqdm(img_loader)):
        optimizer.zero_grad()
        train_pred = model(x_train)
        loss = criterion(train_pred, y_train)
        loss.backward()
        optimizer.step()
        if step % 100 == 0:
            print('Loss: {}'.format(loss))
torch.save(model.state_dict(), 'model-all_data.ckpt')  

  0%|          | 0/8 [00:00<?, ?it/s]

Train data length: 119
Img Shape: torch.Size([1, 28, 28])
Number of Labels: 15
Epoch 0


 12%|█▎        | 1/8 [00:00<00:04,  1.61it/s]

Loss: 1.983608365058899


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

Epoch 14


 12%|█▎        | 1/8 [00:00<00:05,  1.31it/s]

Loss: 2.0672130584716797


100%|██████████| 8/8 [00:06<00:00,  1.28it/s]
